<a href="https://colab.research.google.com/github/praatikgote/Twitter_Sentimental_Analysis/blob/main/Twitter_Sentimental.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
! pip install kaggle

In [ ]:
#configuring the path of Kaggle.json-file
!mkdir ~p/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

mkdir: cannot create directory ‘~p/.kaggle’: No such file or directory


In [ ]:
#api to fetch data set from kaggle
!kaggle datasets download -d kazanova/sentiment140

 79% 64.0M/80.9M [00:00<00:00, 155MB/s]
100% 80.9M/80.9M [00:00<00:00, 166MB/s]


In [ ]:
#extracting compressed zipfile
from zipfile import ZipFile

dataset = '/content/sentiment140.zip'

with ZipFile(dataset , 'r') as zip:

  zip.extractall()
  print('dataset extracted')

dataset extracted


In [ ]:
import numpy as np

import pandas as pd

import re

from nltk.corpus import stopwords

from nltk.stem.porter import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression

from sklearn.metrics import accuracy_score

In [ ]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [ ]:
print(stopwords.words('english'))

['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', '

In [37]:
Twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', error_bad_lines=False)

<ipython-input-37-b467079074c1>:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  Twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO-8859-1', error_bad_lines=False)


In [39]:
#check number of rows and column

Twitter_data.shape

(1599999, 6)

In [40]:
#print the first 5 rows

Twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [42]:
#naming the columns and reading the dataset again

column_name = ['target', 'id', 'date', 'flag', 'user', 'text']

Twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', names=column_name, encoding = 'ISO-8859-1')

In [43]:
Twitter_data.shape

(1600000, 6)

In [44]:
Twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [45]:
#count the missing values in the data set

Twitter_data.isnull().sum()

target    0
id        0
date      0
flag      0
user      0
text      0
dtype: int64

In [46]:
#checking the distribution of target column
Twitter_data['target'].value_counts()

0    800000
4    800000
Name: target, dtype: int64

In [47]:
Twitter_data.replace({'target':{4:1}}, inplace=True)

In [48]:
#chaking the disturbution of the target element

Twitter_data['target'].value_counts()

0    800000
1    800000
Name: target, dtype: int64

**Stemming**

Stemming is the process of reducing of words to its root words

example : actor , actress, aacting  = act




In [50]:
port_stem = PorterStemmer()

In [51]:
def stemming(content):
  stemmed_content = re.sub('[^a-zA-Z]',' ', content)
  stemmed_content = stemmed_content.lower()
  stemmed_content = stemmed_content.split()
  stemmed_content = [port_stem.stem(word) for word in stemmed_content if not word in stopwords.words('english')]
  stemmed_content = ' '.join(stemmed_content)

  return stemmed_content

In [52]:
Twitter_data['stemmed_content'] = Twitter_data['text'].apply(stemming) #it take 50mins

KeyboardInterrupt: 

In [53]:
Twitter_data.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


In [54]:
print(Twitter_data['stemmed_content'])

KeyError: 'stemmed_content'

In [55]:
print(Twitter_data['target'])

0          0
1          0
2          0
3          0
4          0
          ..
1599995    1
1599996    1
1599997    1
1599998    1
1599999    1
Name: target, Length: 1600000, dtype: int64


In [58]:
#seprating the data and label

X = Twitter_data['stemmed_content'].values
Y = Twitter_data['target'].values


KeyError: 'stemmed_content'

In [ ]:
print(X)

In [59]:
print(Y)

NameError: name 'Y' is not defined

splitting the data into training and testion data


In [ ]:
X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, stratify=Y, random_state=2)

In [ ]:
print(X.shape, X_train.shape, X_test.shape)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

In [ ]:
# converting the textual data to numerical data
vectorizer = TfidfVectorizer()

X_train = vectorizer.fit_transform(X_train)

X_test = vectorizer.transform(X_test)

In [ ]:
print(X_train)

In [ ]:
print(X_test)

Training the machine learning models

Logistic Regretion

is a classification model

In [63]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train, Y_train)

Model Evaluation

Accuracy Score



In [ ]:
# accuracy score on the training data

X_train_prediction = model.predict(X_train)

training_data_accuracy = accuracy_score(Y_train, X_train_prediction)

In [ ]:
print('Accuracy Score on training data: ',training_data_accuracy)

In [ ]:
# accuracy score on the test data

X_test_prediction = model.predict(X_test)

testing_data_accuracy = accuracy_score(Y_test, X_test_prediction)

In [ ]:
print('Accuracy Score on testing data: ',testing_data_accuracy)

model accuracy = 77.8%

saving the training model

In [61]:
import pickle

In [62]:


filename = 'trained_model.sav'

pickle.dump(model, open(filename, 'wb'))

NameError: name 'model' is not defined

using the saved model for future prediction


In [ ]:
# loading the saved model

loaded_model = pickle.load(open('/content/trained_model.sav', 'rb'))

In [ ]:
X_new = X_test[200]

print(Y_test[200])

prediction = model.predict(X_new)
print(prediction)

if (prediction[0] == 0):
  print('Negative Tweet')

else:
  print('Positive Tweet')